In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

import PortOpt.invariants as invs
import PortOpt.moment_est as moments
from PortOpt.opt_allocations import Optimiser
import PortOpt.utility_functions as utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load in Data

In [3]:
tickers = ['AAPL', 'MSFT', 'CVX', 'GE', 'GOOGL']

stock_data = utils.get_data(tickers, '1d', '2015-01-01', '2021-01-01')

[*********************100%***********************]  1 of 1 completed
AAPL data downloaded...
[*********************100%***********************]  1 of 1 completed
MSFT data downloaded...
[*********************100%***********************]  1 of 1 completed
CVX data downloaded...
[*********************100%***********************]  1 of 1 completed
GE data downloaded...
[*********************100%***********************]  1 of 1 completed
GOOGL data downloaded...


### Get Expected Returns

In [4]:
stock_returns = invs.stock_invariants(stock_data)
stock_returns.head()

,AAPL,MSFT,CVX,GE,GOOGL
2015-01-02 00:00:00,-0.028576,-0.009238,-0.040792,-0.018527,-0.019238
2015-01-05 00:00:00,0.000094,-0.014786,-0.000463,-0.021780,-0.024989
2015-01-06 00:00:00,0.013925,0.012625,-0.000834,0.000415,-0.002945
2015-01-07 00:00:00,0.037702,0.028994,0.022625,0.011971,0.003478
2015-01-08 00:00:00,0.001072,-0.008441,-0.020127,-0.014050,-0.012286


In [5]:
exp_returns = stock_returns.mean() * 252
exp_returns

AAPL     0.279237
MSFT     0.280026
CVX     -0.004645
GE      -0.111407
GOOGL    0.199739
dtype: float64

### Get Covariance Matrix

In [14]:
riskmodel = moments.RiskModel(tickers)

stock_cov = riskmodel.avg_hist_cov(stock_data)
stock_cov

,AAPL,MSFT,CVX,GE,GOOGL
AAPL,0.087979,0.056670,0.038897,0.038112,0.048528
MSFT,0.056670,0.077349,0.042147,0.037390,0.054574
CVX,0.038897,0.042147,0.104277,0.063808,0.036403
GE,0.038112,0.037390,0.063808,0.140668,0.035049
GOOGL,0.048528,0.054574,0.036403,0.035049,0.071333


### Solve Optimisation Problem

In [15]:
optimiser = Optimiser(tickers, exp_returns, stock_cov)
weights = optimiser.mean_variance(threshold=0.1)
# weights = optimiser.max_sharpe()

# min_pos = lambda x: x >= 0.1*np.ones(len(tickers))
# optimiser.add_constraint(min_pos)

# weights = optimiser.solve(objective='max_sharpe')
weights

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Oct 08 05:07:44 AM: Your problem has 5 variables, 4 constraints, and 0 parameters.
(CVXPY) Oct 08 05:07:44 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 08 05:07:44 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 08 05:07:44 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Oct 08 05:07:44 AM: Compiling problem (target solver=OSQP).
(CVXPY) Oct 08 05:07:44 AM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing 

AAPL     0.182566
MSFT     0.143655
CVX      0.190582
GE       0.117543
GOOGL    0.365654
dtype: float64

In [24]:
optimiser.weight_tearsheet(weights)

Annual Return: 15.026
Annual Volatility: 23.203
Annual Sharpe: 0.561
